In [1]:
import pandas
data_set = pandas.read_csv('data/yelp.csv')
data_set.head(4)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0


In [2]:
data_set.describe()

,stars,cool,useful,funny
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,3.777500,0.876800,1.409300,0.701300
std,1.214636,2.067861,2.336647,1.907942
min,1.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,1.000000,0.000000
75%,5.000000,1.000000,2.000000,1.000000
max,5.000000,77.000000,76.000000,57.000000


In [3]:
x = data_set.text
y = data_set.stars

In [4]:
from sklearn.cross_validation import train_test_split

x_train, x_test, y_train, y_test = train_test_split (x, y, random_state=1)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
print (x_train.shape)
print (x_test.shape)

(7500,)
(2500,)


In [6]:
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#%time cvect = CountVectorizer(stop_words = 'english', max_features = 500)
cvect = TfidfVectorizer(stop_words = 'english', max_features = 2000)
cvect.fit(x_train)
%time x_train_dtm = cvect.transform(x_train)
x_train_dtm.shape

x_train_dtm

CPU times: user 808 ms, sys: 0 ns, total: 808 ms
Wall time: 807 ms


<7500x2000 sparse matrix of type '<type 'numpy.float64'>'
	with 289284 stored elements in Compressed Sparse Row format>

In [7]:
x_test_dtm = cvect.transform(x_test)

In [8]:
from sklearn import svm
clf = svm.LinearSVR()
clf.fit(x_train_dtm, y_train)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

In [9]:
y_test_pred = clf.predict(x_test_dtm)

In [22]:
y_test_pred_int = [0] * y_test_pred.size
i = 0
for y in y_test_pred:
    y_test_pred_int[i] = (int(y+0.5))
    if y_test_pred_int[i] > 5:
        y_test_pred_int[i] = 5
    if y_test_pred_int[i] < 1:
        y_test_pred_int[i] = 1
    i += 1

In [23]:
from sklearn import metrics

metrics.accuracy_score(y_test, y_test_pred_int)

0.44800000000000001

In [24]:
print (metrics.classification_report(y_test, y_test_pred_int))

             precision    recall  f1-score   support

          1       0.67      0.12      0.20       185
          2       0.37      0.27      0.31       234
          3       0.27      0.43      0.33       365
          4       0.46      0.62      0.53       884
          5       0.62      0.40      0.48       832

avg / total       0.49      0.45      0.44      2500



In [25]:
metrics.confusion_matrix(y_test, y_test_pred_int)

array([[ 22,  60,  72,  31,   0],
       [  8,  63, 121,  41,   1],
       [  3,  29, 157, 150,  26],
       [  0,  15, 149, 548, 172],
       [  0,   3,  75, 424, 330]])